In [ ]:
!ffmpeg

In [ ]:
from IPython.display import display, Image, HTML

In [ ]:
import ipywidgets as widgets

In [ ]:
from flat import document, shape, rgb

In [ ]:
from IPython.display import SVG, display
def show(page):
    display(SVG(page.svg()))

In [ ]:
def print_val(step=0):
    print(step)

In [ ]:
play_widget = widgets.Play(min=0, max=30, interval=100)
widgets.interact(print_val, step=play_widget)

In [ ]:
def render(step=0):
    page = document(80, 80, 'mm').addpage()
    page.place(shape().nostroke().fill(rgb(255, 255, 255)).rectangle(0, 0, 80, 80))
    page.place(shape().nostroke().fill(rgb(40, 40, 160)).ellipse(40+step, 40, step, step*1.5))
    return page

In [ ]:
show(render(30))

In [ ]:
show(render(15))

In [ ]:
def render_and_show(step=0):
    show(render(step))

In [ ]:
widgets.interact(render_and_show, step=widgets.Play(min=0, max=30, interval=10))

In [ ]:
page = render(10)
png_bytes = page.image(ppi=72, kind='rgb').png()
png_bytes[:100]

In [ ]:
def page2png(page, dpi=72):
    try:
        import cairosvg
        png_bytes = cairosvg.svg2png(page.svg(), dpi=dpi)
    except ModuleNotFoundError:
        png_bytes = page.image(ppi=dpi, kind='rgb').png()
    return png_bytes

In [ ]:
png_data = page2png(render(10))

In [ ]:
png_data[:100]

In [ ]:
display(Image(png_data))

In [ ]:
!mkdir render

In [ ]:
with open("render2/test.png", "wb") as fh:
    fh.write(page2png(render(10)))

In [ ]:
for i in range(10):
    fname = f"render/image{i:05}.png"
    print(fname)

In [ ]:
for i in range(30):
    fname = f"render/image{i:05}.png"
    png_data = page2png(render(i))
    with open(fname, "wb") as fh:
        fh.write(png_data)

In [ ]:
!ls render

In [ ]:
!open render

In [37]:
!ffmpeg -loglevel warning -y \
    -framerate 30 -f image2 -i render/image%05d.png \
    -vcodec libx264 -crf 10 -pix_fmt yuv420p render2/output.mp4

To display the video in the notebook:

In [38]:
from IPython.display import HTML
display(HTML("""
    <video alt="rendered output" controls>
        <source src="render/output.mp4" type="video/mp4">
    </video>
"""))

In [ ]:
!ffmpeg -loglevel warning -y \
    -framerate 30 -f image2 -i render/image%05d.png \
    -filter_complex "[0:v] split [a][b];[a] palettegen [p];[b][p] paletteuse" render2/output.gif

In [ ]:
display(Image(open("render2/output.gif", "rb").read()))

In [ ]:
from bezmerizing import Polyline, Path
from copy import copy
import numpy as np

In [ ]:
from flat import font

In [ ]:
f = font.open("NotoSans-Regular.ttf")

In [ ]:
def glyphcommands(f, ch):
    return Path([copy(cmd) for cmd in f.glyph(f.charmap[ord(ch)])])
def advancefor(f, ch):
    return f.advances[f.charmap[ord(ch)]]

In [ ]:
s = "x o x o x o x o x o x o x o x o x o x o"

In [ ]:
end_frame = 600
def render(step=0):
    np.random.seed(12345)
    page = document(500, 500, 'pt').addpage()
    bg_pen = shape().nostroke().fill(rgb(255-step, 255-step*2, step*3))
    page.place(bg_pen.rectangle(0, 0, 300*step/100, 90*step/100))
    pen = shape().nostroke().fill(rgb(step, 0, 0))
    factor = 36 / f.density
    cx = 0
    fraction = 1 - (step / end_frame)
    for x in range(60):
        for ch in s:
            glyph_path = (glyphcommands(f, ch)
                          .scale(factor)
                          .translate(cx + (np.random.normal(step, 30) * fraction * x),
                                     60 + (np.random.normal(x*100, 30) * fraction * x)))
            page.place(pen.path(glyph_path))
            
        cx += advancefor(f, ch) * factor / 100
    return page

In [ ]:
show(render(20))

In [ ]:
widgets.interact(lambda step: show(render(step)),
                 step=widgets.Play(min=0, max=end_frame, interval=1000/30))

In [ ]:
for i in range(end_frame):
    fname = f"render/image{i:05}.png"
    png_data = page2png(render(i))
    with open(fname, "wb") as fh:
        fh.write(png_data)

In [ ]:
!ffmpeg -loglevel warning -y \
    -framerate 30 -f image2 -i render/image%05d.png \
    -vcodec libx264 -crf 10 -pix_fmt yuv420p render/output.mp4

In [ ]:
from sklearn.cluster import MiniBatchKMeans

In [ ]:
def kmeans_glyph(n_clusters=6):
    data = np.random.uniform(low=-0.5, high=0.5, size=(500, 2))
    km = MiniBatchKMeans(n_clusters=n_clusters, max_iter=25)
    km.fit(data)
    np.random.shuffle(km.cluster_centers_)
    return Polyline(km.cluster_centers_)

In [ ]:
glyph = kmeans_glyph(10).scale(100).translate(50, 50)
glyph_poly = glyph.fancy_curve(samples_per=24,
                               thicknesses=[2.5, 1.5, 0.5, 2],
                               tightness=-0.5)
page = document(100, 100, 'mm').addpage()
brush = shape().nostroke().fill(rgb(40, 40, 40))
page.place(brush.polygon(glyph_poly))
show(page)

In [ ]:
n_frames = 25
glyph_a = kmeans_glyph(10).scale(100).translate(50, 50)
glyph_b = kmeans_glyph(10).scale(100).translate(50, 50)
interp = np.linspace(glyph_a.vertices, glyph_b.vertices, n_frames+1)
def render(step=0):
    this_glyph = Polyline(interp[step])
    glyph_poly = this_glyph.augment().fancy_curve(samples_per=24,
                               thicknesses=[2.5, 1.5, 0.5, 2],
                               tightness=-0.5)
    page = document(100, 100, 'mm').addpage()
    brush = shape().nostroke().fill(rgb(40, 40, 40))
    page.place(brush.polygon(glyph_poly))
    return page

In [ ]:
widgets.interact(lambda step: show(render(step)),
                 step=widgets.Play(min=0, max=n_frames, step=1, interval=1000/30))

In [ ]:
from itertools import chain

# canvas
width = 250
height = 250
# number of interpolations and steps per interpolation
n_interps = 12
n_steps = 24
interps = []
# generate interpolations
start = kmeans_glyph(10)
current = start
for i in range(n_interps):
    end = kmeans_glyph(10)
    interp = np.linspace(current.vertices, end.vertices, n_steps)
    # make the last value of this interpolation the first of the next
    current = end 
    interps.append(interp)
# loop back to first
interps.append(np.linspace(current.vertices, start.vertices, n_steps))
# flatten list of lists
interps = list(chain(*interps))

def render(step=0):
    this_glyph = Polyline(interps[step]).scale(width).translate(width*0.5, height*0.5)
    glyph_poly = this_glyph.augment().fancy_curve(samples_per=24,
                               thicknesses=[5, 3, 1, 4],
                               tightness=-0.5)
    page = document(width, height, 'mm').addpage()
    background = shape().nostroke().fill(rgb(255, 255, 255))
    brush = shape().nostroke().fill(rgb(40, 40, 40))
    page.place(background.rectangle(0, 0, width, height))
    page.place(brush.polygon(glyph_poly))
    return page

In [ ]:
widgets.interact(lambda step: show(render(step)),
                 step=widgets.Play(min=0, max=len(interps)-1, step=1, interval=1000/30))

In [ ]:
print(len(interps))
for i in range(len(interps)):
    print(i, end=" ")
    fname = f"render/image{i:05}.png"
    png_data = page2png(render(i))
    with open(fname, "wb") as fh:
        fh.write(png_data)

In [39]:
!ffmpeg -loglevel warning -y \
    -framerate 30 -f image2 -i render/image%05d.png \
    -vcodec libx264 -crf 10 -pix_fmt yuv420p render/output.mp4

In [ ]:
import random
import cairosvg
%timeit cairosvg.svg2png(render(random.randrange(0, 30)).svg(), dpi=72)

In [ ]:
%timeit render(random.randrange(0, 30)).image(ppi=72, kind='rgb').png()